In [ ]:

import warnings
warnings.filterwarnings('ignore')
from ir_measures import AP, nDCG, P, R, RR, MRR
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
import numpy as np
import os
import shutil

work_name="retrievability-bias"
root_dir = f'/root/{work_name}'
nfs_dir = f'/nfs/datasets/cxj/{work_name}'
if not os.path.exists(nfs_dir):
    os.makedirs(nfs_dir)

dataset_name = 'msmarco-passage'
eval_ds_name = 'trec-dl-2019'
text_field= 'text'
ranker = "BM25"
retrieve_num = 10
dataset = pt.get_dataset(f'irds:{dataset_name}')
eval_dataset = pt.get_dataset(f'irds:{dataset_name}/{eval_ds_name}')
topics = eval_dataset.get_topics()
qrels = eval_dataset.get_qrels()

# dataset_index = f'/nfs/llm/indices/{dataset_name}'
# index = pt.IndexRef.of(f'{dataset_index}')
index = pt.IndexFactory.of(f'/nfs/llm/indices/{dataset_name}/data.properties')

index.getCollectionStatistics

In [ ]:
import pyterrier as pt
if not pt.java.started():
    pt.java.init()

In [ ]:
dataset = pt.get_dataset(f'irds:msmarco-passage')
eval_dataset = pt.get_dataset(f'irds:msmarco-passage/dev')
topics = eval_dataset.get_topics()
topics.shape[0]

In [ ]:
if os.path.exists('./df_msmarco.csv'):
    df = pd.read_csv('./df_msmarco.csv')
else:
    df = pd.DataFrame(dataset.get_corpus_iter(verbose=True))
    df.to_csv('./df_msmarco.csv', index=True)

In [ ]:
# sample_docs = df[text_field].sample(n=1000,random_state=1)
sample_df = df.sample(n=1000000,random_state=1)
print(sample_df[0:5])

In [ ]:
def is_number(s):
    try:  
        float(s)
        return True
    except ValueError:  
        pass  
    try:
        import unicodedata  
        unicodedata.numeric(s)  
        return True
    except (TypeError, ValueError):
        pass
    return False

In [ ]:
di = index.getDirectIndex()
doi = index.getDocumentIndex()
lex = index.getLexicon()

In [ ]:
Q = []
terms = []
count=0
for docid, _ in sample_df.iterrows():
    # print(docid)
    # print(f'terms={terms}')
    for posting in di.getPostings(doi.getDocumentEntry(docid)):
        termid = posting.getId()
        lee = lex.getLexiconEntry(termid)
        term = lee.getKey()
        freq = posting.getFrequency()
        if freq >=2 and not is_number(term):
            # print("%s with frequency %d" % (term,freq))
            terms.append(term)
            if len(terms) == 3:
                Q.append(' '.join(terms))
                terms = []
    
    terms = []
    # count +=1
    # if count ==8:
    #     break

In [7]:
print(f'Q={len(Q)}')

Q=110276


In [8]:
Q[:10]

['school letter offici',
 'will cost averag',
 'ft sq feet',
 'offic bui polic',
 'complex person xenophobia',
 'process includ death',
 'water pump machin',
 'wash drain tub',
 'truste transfer ira',
 'onc grow plant']

In [ ]:
df_Q = pd.DataFrame({'qid':range(0,len(Q)),'query':Q})

In [20]:
df_Q.shape

(1363781, 3)

In [2]:
# df_Q.to_csv('./df_Q.csv')
df_Q = pd.read_csv('./df_Q.csv')

In [3]:
# tf_idf = pt.terrier.Retriever(index, wmodel="TF_IDF")
bm25 = pt.terrier.Retriever(index, wmodel="BM25") % 10

In [4]:
# pipe = bm25 >> pt.text.get_text(dataset,'text')

In [5]:
sample_Q = df_Q.sample(n=100000,random_state=1)

In [7]:
result = bm25.transform(sample_Q)

In [17]:
result.shape

(999744, 7)

In [12]:
result.to_csv('./result.csv')

In [ ]:
result.groupby(['docid']).apply(lambda x:x)

In [ ]:
g = result.groupby(['docid'])['docid'].count()
df_g = pd.DataFrame(g)
df_g.index.name='inx'
df_g.sort_values(by='docid',ascend=False)

In [ ]:
df_g

In [23]:
csv = '/nfs/primary/retrievability-bias/results/new_clustered/clustered_dev_queries_by_10000.csv'
df = pd.read_csv(csv, index_col=0).reset_index()

In [24]:
df = df.sort_values('cluster')

In [25]:
df.head(100)

,qid,query,cluster
7788,439876,lifespan treated lumber,0
51838,646071,what does preferred provider organization stan...,0
90723,825988,what is the industry standard organization for...,0
90720,825986,what is the industry standard for machine fini...,0
17783,1086360,what can cause vomiting liquid but not your food,1
17496,1086023,what causes vomiting seizure and blood drippin...,1
12717,816254,what is the difference between a clear diet an...,1
77875,731172,what is clear liquid that runs from bites,1
21000,1090121,targetize app,2
3444,1056339,who was the founder of target,2


In [12]:
df2 = df.groupby(by='cluster')

In [18]:
df2.get_group(300).shape

(65, 3)